<div style="width:100%;height:125px;text-align:center;border: 4px solid #F2E203;background-color:#F2E203;color:black">

<header  style="width:100%;height:100px;">
  <h1><b>Chapter 8.1</b></h1>
    <h4>TensorFlow and L1/L2 - part2</h4>
</header>

<div> 

<div style="position: relative;height:100px;">
    
<div  style="width:300px;position:absolute;left: auto;border: 4px solid white;background-color:#F2E203;color:black">
    <header></header>
    <a href="#t" style="padding:5px;color:black;text-align: center;" href="#Chapter3">
      <h4 ><b>TensorFlow and L1/L2</b></h4>
      </a>
 </div>
    
 <div style="width:300px;position:absolute;left: 305px;border: 4px solid white;background-color:#F2E203;color:black">
    <header></header>
    <a href="#data"style="position: relative;padding:5px;color:black;;text-align: center;" href="#Chapter3">
      <h4 ><b>data</b></h4>
      </a>
     
  </div>
    <div style="width:300px;position:absolute;left: 610px;border: 4px solid white;background-color:#F2E203;color:black">
    <header></header>
    <a href="#model"style="position: relative;padding:5px;color:black;;text-align: center;" href="#Chapter3">
      <h4 ><b>Model</b></h4>
      </a>
        </div>

    
   <div  style="width:300px;position:absolute;left: 915px;border: 4px solid white;background-color:#F2E203;color:black">
    <header></header>
    <a href="#list"style="position: relative;padding:5px;color:black;;text-align: center;" href="#Chapter3">
      <h4 ><b>load data</b></h4>
      </a>
    </div>
  
    
   <div  style="width:300px;position:absolute;left: 1220px;border: 4px solid white;background-color:#F2E203;color:black">
    <header></header>
    <a href="#list"style="position: relative;padding:5px;color:black;text-align: center;" href="#Chapter3">
      <h4 ><b></b></h4>
      </a>
    </div>
</div>
<!------------------------------------------------------------------------------------------------------------------------>


<div id="t" style="width:100%;height:70px;border: 4px solid #F2E203;background-color:#F2E203;color:white;text-align:center;border-radius: 25px;padding:3px;color:black">
  <header><h2><b>TensorFlow and L1/L2</b></h2></header>
</div>

<div style="border: 4px solid #BFE6FF;background-color:#BFE6FF;color:black;border-radius: 5px;padding:7px">
  <strong> Refrence: </strong><br>
    

https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class09_regularization.ipynb
    
https://www.youtube.com/watch?v=xyymDGReKdY&index=31&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&frags=wn   
    
</div>

Both L1 and L2 work differently in the way that they penalize the size of a weight. L2 will force the weights into a pattern similar to a Gaussian distribution; the L1 will force the weights into a pattern similar to a Laplace distribution, as demonstrated the following:
<img src="https://camo.githubusercontent.com/ace7a6311e1b7c67fda7425a6a9c74627335b084/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6a656666686561746f6e2f7438315f3535385f646565705f6c6561726e696e672f6d61737465722f696d616765732f636c6173735f395f6c315f6c322e706e67">

 L1 algorithm is more tolerant of weights further from 0, whereas the L2 algorithm is less tolerant. We will highlight other important differences between L1 and L2 in the following sections. You also need to note that both L1 and L2 count their penalties based only on weights; they do not count penalties on bias values.
 
Tensor flow allows [l1/l2 to be directly added to your network](https://tensorlayer.readthedocs.io/en/stable/modules/cost.html).

<div id="data" style="width:100%;height:70px;border: 4px solid #F2E203;background-color:#F2E203;color:white;text-align:center;border-radius: 25px;padding:3px;color:black">
  <header><h2><b>data</b></h2></header>
</div>

In [1]:
import pandas as pd
df = pd.read_csv("data/auto-mpg.csv",na_values=["NA","?"])
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite


In [2]:
from farhad_DL.utility import missing_median, to_xy
from sklearn.model_selection import train_test_split

In [3]:
from farhad_DL.utility import missing_median
df.drop('name',axis=1,inplace=True)
missing_median(df, "horsepower")
x, y = to_xy(df, 'mpg')
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state= 42)

<div id="model"style="width:100%;height:70px;border: 4px solid #F2E203;background-color:#F2E203;color:white;text-align:center;border-radius: 25px;padding:3px;color:black">
  <header><h2><b>Model</b></h2></header>
</div>

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.logging import set_verbosity, ERROR

from tensorflow.keras import regularizers 

from sklearn.metrics import confusion_matrix, mean_squared_error
from farhad_DL.utils import TimeSummary, plot_training_summary, Estimate_Deeptime

import numpy as np

In [21]:
def Regularization():
    model = Sequential()
    
    model.add(Dense(50, input_dim=x.shape[1],activation="relu"))
    model.add(Dense(25,activation="relu"))
    model.add(Dense(10,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="relu"))
    
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
    return model

In [27]:
set_verbosity(ERROR)
epochs = 1000
monitor = EarlyStopping(monitor='val_cross',mode='auto',verbose=1,min_delta=1e-3,patience=5)
#estimate_cpu = Estimate_Deeptime(epochs)
time_summary = TimeSummary()

model = Regularization()
summary = model.fit(x_train,y_train, validation_data=(x_test,y_test), 
          callbacks=[monitor,time_summary], verbose=0,epochs=epochs)

pred = model.predict(x_test)
rm = np.sqrt(mean_squared_error(pred, y_test))
score = model.evaluate(x_test, y_test)
print("Final score (RMSE):",rm)
print("Final Losse:", score[0])
print("Final accuracy:",score[1])
#plot_training_summary(time_summary,summary)

100/100 [==============================] - 0s 47us/step
Final score (RMSE): 2.561331
Final Losse: 16.036672039031984
Final accuracy: 0.11
